In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys
from tqdm import tqdm

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
from glob import glob
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

In [ ]:
d = {
    'x_res'         : [400],
    'y_res'         : [800],
    'nr_forw_lines' : [50],
    'nr_back_lines' : [50],
    'nr_stat_lines' : [10],
    'forw_intens'   : [10.0],
    'back_intens'   : [10.0],
    'stat_intens'   : [10.0],
    'forw_speed'    : [1.0],
    'back_speed'    : [1.0],
    'forw_width'    : [9.0],
    'back_width'    : [9.0],
    'stat_width'    : [1.0],
    'forw_speed_var': [0.3],
    'back_speed_var': [0.3],
    'forw_width_var': [1.0],
    'back_width_var': [2.0],
    'stat_width_var': [2.0],
}

dataframe = pd.DataFrame(d)

nr_tests = 10

expo_linspace = np.array( np.linspace(1, 1000, nr_tests))

dataframe.loc[0, 'nr_forw_lines'] = int(expo_linspace[0])
dataframe.loc[0, 'nr_back_lines'] = int(1001 - expo_linspace[0] )
# dataframe.loc[0, 'forw_speed_var'] = 1.0

for i in range(nr_tests-1):
    dataframe.loc[len(dataframe.index)] = dataframe.iloc[0]
    dataframe.loc[i+1, 'nr_forw_lines'] = int(expo_linspace[i+1])
    dataframe.loc[i+1, 'nr_back_lines'] = int(1001 - expo_linspace[i+1])
#     dataframe.loc[i+1, 'forw_speed_var'] = (9-i)/10

dataframe['nr_forw_lines'] = np.array(np.random.uniform(20, 200, nr_tests), dtype=int)
dataframe['nr_back_lines'] = np.array(np.random.uniform(20, 200, nr_tests), dtype=int)
dataframe['nr_stat_lines'] = np.array(np.random.uniform(1, 30, nr_tests), dtype=int)
dataframe['forw_intens']   = np.array(abs(np.random.normal(10, 4, nr_tests)))
dataframe['back_intens']   = np.array(abs(np.random.normal(10, 4, nr_tests)))
dataframe['stat_intens']   = np.array(abs(np.random.normal(10, 4, nr_tests)))
dataframe['forw_speed']    = np.array(np.random.normal(1.5, 1.5, nr_tests))
dataframe['back_speed']    = np.array(np.random.normal(1.5, 1.5, nr_tests))
dataframe['forw_width']    = np.array(np.random.normal(17, 5, nr_tests))
dataframe['back_width']    = np.array(np.random.normal(17, 5, nr_tests))
dataframe['stat_width']    = np.array(np.random.normal(17, 5, nr_tests))
dataframe['forw_speed_var']    = abs(np.array(np.random.normal(0.5, 0.2, nr_tests)))
dataframe['back_speed_var']    = abs(np.array(np.random.normal(0.5, 0.2, nr_tests)))
dataframe['forw_width_var']    = abs(np.array(np.random.normal(3, 1, nr_tests)))
dataframe['back_width_var']    = abs(np.array(np.random.normal(3, 1, nr_tests)))
dataframe['stat_width_var']    = abs(np.array(np.random.normal(3, 1, nr_tests)))

test_title = f"Scatterplot of transport tests"
test_xaxis = "Ground Truth Transport"

print(test_title)

dataframe


In [ ]:
directory_targ = '/gpfs/home6/svstaalduine/Huh/'

In [ ]:
run_parallel(
    "line_test_slurm.py",
    [directory_targ, 10],
    dataframe,
    32,
    "10:00:00",
    "line_test"
)

In [ ]:
plot_range= 3000

dataframe = pd.read_json("/gpfs/scratch1/shared/amftrackflow/temp/1682321952951926320.json")


In [ ]:

files_list = glob(os.path.join(directory_targ, '1682321952951926320_*_line_test.json'))

import matplotlib.cm as cmx

total_lines = dataframe['nr_forw_lines'] * dataframe['forw_intens'] + dataframe['nr_back_lines'] * dataframe['back_intens']
line_group = total_lines // 100
total_spd_var = dataframe['forw_speed_var'] + dataframe['back_speed_var']
total_width_var = dataframe['forw_width_var'] + dataframe['back_width_var'] + dataframe['stat_width_var']
line_ratio = (dataframe['nr_forw_lines'] * dataframe['forw_intens']) / (dataframe['nr_back_lines'] * dataframe['back_intens'])
speed_ratio = dataframe['forw_speed'] / dataframe['back_speed']
high_speed = dataframe['back_speed'] > 2
test_index = np.arange(nr_tests)

line_means = dataframe.groupby(line_group).mean()
print(line_means)

value_interest = total_lines
cbar_string = 'Total number of lines times intensity'


In [ ]:
print(max(line_group))

In [ ]:
# print(total_lines.max())
jet = cm = plt.get_cmap('viridis') 
cNorm  = colors.Normalize(vmin=value_interest.min(), vmax=value_interest.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

# print(len(files_list))
fig, ax = plt.subplots(2, figsize = (6,6*2))

for i, file in enumerate(tqdm(sorted(files_list))):  
    file_int = int(file.split('/')[-1].split('_')[-3])
#     print(int(file.split('/')[-1].split('_')[-3]))
    res = pd.read_json(file)
    
    colorVal = scalarMap.to_rgba(value_interest[i])
    
    if i == 0:
        est_res = pd.DataFrame(res.iloc[0]).T
        grt_res = pd.DataFrame(res.iloc[1]).T
        histo_counts, histo_bins = np.histogram((res.iloc[0] - res.iloc[1]), bins = 50, range=(0, 2000))
    else:
        est_res = pd.concat([est_res, pd.DataFrame(res.iloc[0]).T], ignore_index=True)
        grt_res = pd.concat([grt_res, pd.DataFrame(res.iloc[1]).T], ignore_index=True)
        histo_count, _ = np.histogram((res.iloc[0] - res.iloc[1]), bins = 50, range=(0, 2000))
        histo_counts += histo_count
    error_frame = abs(res.iloc[0] - res.iloc[1]).T
    
    ax[0].scatter(pd.DataFrame(res.iloc[1]).T, pd.DataFrame(res.iloc[0]).T, s=3, color=colorVal, alpha=0.05)
    ax[1].scatter(pd.DataFrame(res.iloc[1]).T, error_frame, s=3, color=colorVal, alpha=0.1)
#     ax[3].scatter([total_lines[i]]*10, error_frame)
ax[0].plot([-plot_range, plot_range], 
        [-plot_range, plot_range], label='1:1', linestyle='dashed')
ax[0].set_xlim(-plot_range, plot_range)
ax[0].set_ylim(-plot_range, plot_range)
ax[0].legend()
ax[0].grid(True)
ax[0].set_title(test_title)
ax[0].set_xlabel(test_xaxis)
ax[0].set_ylabel("Estimated Transport")
cbar = fig.colorbar(scalarMap, ax=ax[0])
cbar1 = fig.colorbar(scalarMap, ax=ax[1])
cbar.set_label(cbar_string, rotation=270, labelpad = 12)
cbar1.set_label(cbar_string, rotation=270, labelpad = 12)
ax[1].set_xlabel('Ground truth transport')
ax[1].set_ylabel('Transport Error')
ax[1].set_title('Error plot')



In [ ]:
fig, ax = plt.subplots(2, figsize = (8,8*1.5))

line_group = total_lines // 100
total_speed = dataframe['forw_speed'] + dataframe['back_speed']
total_speed_group = total_speed // 1

total_var = dataframe['forw_speed_var'] + dataframe['back_speed_var']
total_var_group = total_var * 10 // 1

error_frames = abs(grt_res - est_res)
line_group_means = error_frames.groupby(line_group).mean().mean(axis=1)
line_group_stds = error_frames.groupby(line_group).std().mean(axis=1)

total_speed_group_means = error_frames.groupby(total_speed_group).mean().mean(axis=1)
totat_speed_group_stds = error_frames.groupby(total_speed_group).std().mean(axis=1)

total_var_group_means = error_frames.groupby(total_var_group).mean().mean(axis=1)
total_var_group_stds = error_frames.groupby(total_var_group).std().mean(axis=1)


means_interest = total_var_group_means
std_interest = total_var_group_stds

ax[0].errorbar(means_interest.index, means_interest, std_interest, fmt='o')

ax[1].stairs(histo_counts, histo_bins)

# x_axis = (dataframe['nr_forw_lines'] / dataframe['nr_back_lines']).to_numpy()
# x_axis = dataframe['nr_forw_lines'].to_numpy()*2
estim = est_res.to_numpy()
grdtr = grt_res.to_numpy()
# print(estim)

# ax[0].errorbar(x_axis, est_res.mean(axis=1), yerr=est_res.std(axis=1), label='Estimated', fmt='o')
# ax.scatter(grdtr.T, estim.T)
# ax[0].errorbar(x_axis, grt_res.mean(axis=1), yerr=grt_res.std(axis=1), label='Ground Truth', fmt='o')
# # ax.set_xscale('log')

ax[1].set_xlabel('Transport error')
ax[1].set_ylabel('Frequency')
ax[1].set_title('Error histogram')
ax[0].set_xlabel('Total variance / 10 (forw + back)')
ax[0].set_ylabel('Transport error')
ax[0].set_title('Line amount vs error')
# fig.suptitle("Transport total line amount")
fig.tight_layout()


print(line_group_stds.index)
